# Find noisy and 'oversensitive' pixels

This notebook can be used to detect noisy or oversensitive pixels on the area detector. 
It uses a background measurement of either air or an amorphous sample collected with a timescan - preferably with many exposures.

#### __Use at your own risk:__ _This method is not using strignet statistics, nor has not been validated with different energies, exposure times, number of frames etc._ 


<div class="alert alert-block alert-info">
<b>These parameter values seems to work:</b><br>
<tt>filter_value</tt> = 5 <br>
<tt>n_sigma</tt> = 2 <br>
<tt>cut_off_percent</tt> = 0.1 <br>
<tt>neighbour_intensity_ratio</tt> = 1.1<br><br>
<b>For masking based on geometry, these parameter values seems to work:</b><br> 
<tt>grow_edge_by</tt> = 2<br>
<tt>mask_edge_by</tt> = 2<br>  
<tt>grow_cdte_gap_by</tt> = 1<br>
<tt>maskReadout</tt> = True<br>
<tt>grow_readout_by</tt> = 1<br>
</div>

The following algorithm is applied:
1) Calculate the average image (`avg_img`) and plot it to check that the data is sensible for this purpose: it should not have any sharp features!
2) 
    a) Filtering of data to remove alpha bombs: This is done by comparing a pixel values to the average count in the pixel (from `avg_img`). If it is more than `filter_value` times the average it is considered an outlier and is replaced with the average value (in order not to skew the statistics in the next step!).<br>
    b) Calculation of a mask based on the pixel statistics. All pixels in all exposures are compared to the average value +/- `n_sigma` times the estimated deviation (that is the square root of the average value). The number of exposures where the count is outide this range is summed and compared to `cut_off_percent` times number of exposures. If the number of values outside the range is over this threshold it will be masked.<br>
    c) Calculation of the standard deviation (using `np.std` method) divided by the average image (for diagnostic purposes).
3) Plot the logarithm of the average image (`avg_img`), the mask (`mask`) and the standard deviation/average image. The plots are used for diagnostic purposes:<br>
    a) Checking that pixels standing out in the average image are indeed masked by comparing the left and middle images.<br>
    b) Checking that alpha bomb filtering worked well in right image. If the threshold was correct no high values should be visible.
4) Mask 'oversensitive' that have higher intensites than their neighbours. Here a map of the average neighbouring values are calculated using a 3x3 kernel (with 1/8 in the perimiter and 0 in the central position) over the average image. The average map is then compared to the average neighbour map. If the value of a pixel is higher than the value of `neighbour_intensity_ratio` it is masked. The resulting map can be checkked in the plots.
5) It is possible to investigate the behaviour of a single pixel by inputting the coordinates `px_x, px_y`. The resulting plot shows the number of counts in the series of exposures and the selected 'sigma' range.
6) Mask out additional pixels based on geometry (edges, inter-ASIC lines etc) - similar to _MaskTool_
7) Save an `.npy` mask to be used for integration - remember the beam-stop is not masked in this tool!



#### Load and display avarage data

In [5]:
#fname = '/data/visitors/danmax/20210940/2022042008/raw/Empty_200422_FS_SDD177/scan-1452.h5'
fname = '/data/visitors/danmax/20220402/2022101908/raw/glass_slide/scan-3804.h5'

plot_average = True
#------------------------------------------------------------------------

%matplotlib widget
import h5py
import numpy as np
import matplotlib.pyplot as plt
import os.path
import time
from ipywidgets import IntProgress

start = time.perf_counter()
fh = h5py.File(fname, 'r')

path, f = os.path.split(fname)

scanCmd = fh['entry/title'][()].decode('utf-8').split()
if plot_average:   
    images_h5 = fh['/entry/instrument/pilatus/data']
    images = np.copy(images_h5)
    avg_img = np.mean(images, axis=0)
    
    print(f'The file contains {images.shape[0]} images.')
    print(f'The maximum count in a single frames is {np.max(images)}.')
    plt.figure(figsize=(4,4))
    plt.imshow(np.log(avg_img+3), interpolation='Nearest')
    plt.title('log(average intensity)')
    print(f'File loaded and averaged in {time.perf_counter()-start:.1f} s.\n')


The file contains 301 images.
The maximum count in a single frames is 5166.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

File loaded and averaged in 4.0 s.



#### Filter data and calculate mask based on pixel data

In [6]:
filter_value = 5 # Threshold for outlier rejection - i.e. how many times the average count constitutes an outlier
n_sigma = 2 # Confidence interval used in test
cut_off_percent = 0.1 #0.1 will mark a pixel as bad if 10% of counts falls outside the n_sigma window

# Number of cold and non-existing pixels for pixel-book-keeping
neg_px_mask = np.where(avg_img <= 0, 1, 0)
num_neg_px = np.sum(neg_px_mask)

# Change negative values to np.nan
avg_img = np.where(avg_img <= 0, np.nan, avg_img)

start = time.perf_counter()
print('Step 1/3: Filtering outliers...')
progress = IntProgress(min=0, max=images.shape[0])
display(progress)
for i in range(images.shape[0]):
    images[i,:,:] = np.where(images[i,:,:]>filter_value*avg_img, avg_img, images[i,:,:]) 
    progress.value = i

print(f'Filtering done in {time.perf_counter()-start:.1f} s.\n')

mask3d = np.zeros(images.shape)
sqrt_avg_img = np.sqrt(avg_img)

start = time.perf_counter()
print('Step 2/3: Calculating mask based on pixel statistics...')
progress = IntProgress(min=0, max=images.shape[0])
display(progress)

for n in range(images.shape[0]):
    dev_from_mean = np.abs(images[n,:,:]-avg_img)  
    mask3d[n,:,:] = np.where(n_sigma*sqrt_avg_img > dev_from_mean, 0, 1)
    progress.value = n    
    
# Sum all outliers from the 3D volume to a single frame and check if number of outliers are above the threshold
sum_mask = np.sum(mask3d, axis=0) 
mask = np.where(sum_mask > cut_off_percent*images.shape[0], 1, 0)
print(f'Mask calculation done in {time.perf_counter()-start:.1f} s.\n')

# Total number of masked pixels at this step for pixel-book-keeping
masked_px = np.sum(mask)

print(f'Using a criteria of {n_sigma} sigma and {100*cut_off_percent:.1f}% outliers, {np.sum(mask)-num_neg_px} pixels was masked.\n')

start = time.perf_counter()
print('Step 3/3: Calculating standard deviation map...')
progress = IntProgress(min=0, max=1) # Include progress bar for consistency
display(progress)
stdev_img = np.std(images, axis =0)/avg_img
progress.value = 1
print(f'Standard deviation map calculation done in {time.perf_counter()-start:.1f} s.\n')

Step 1/3: Filtering outliers...


IntProgress(value=0, max=301)

Filtering done in 3.3 s.

Step 2/3: Calculating mask based on pixel statistics...


IntProgress(value=0, max=301)

Mask calculation done in 7.3 s.

Using a criteria of 2 sigma and 10.0% outliers, 300 pixels was masked.

Step 3/3: Calculating standard deviation map...


IntProgress(value=0, max=1)

Standard deviation map calculation done in 4.1 s.



#### Display average data, mask and standard deviation

In [7]:
plt.figure(figsize=(10,4))
ax1 = plt.subplot(1,3,1)
ax1.imshow(np.log(avg_img+3), interpolation='Nearest')
plt.title('log(average intensity)')
ax2 = plt.subplot(1,3,2, sharex=ax1, sharey=ax1)
ax2.imshow(mask, interpolation='Nearest')
plt.title('mask')
ax3 = plt.subplot(1,3,3, sharex=ax1, sharey=ax1)
ax3.imshow(stdev_img, vmin=0, vmax=0.25*np.nanmax(stdev_img), interpolation='Nearest')
plt.title('"standard deviation"')
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

#### Mask oversensitive pixels based on neighbouring values

In [8]:
neighbour_intensity_ratio = 1.1 # Rejection threshold for ratio of pixel intensities to  average neighbour intensities

import scipy.ndimage as ndi

# Kernel to find intensity average of neighbouring pixels
neighbour_kernel = np.array([[1/8, 1/8, 1/8],
                             [1/8,   0, 1/8],
                             [1/8, 1/8, 1/8]])

# Calc intensity average of neighbouring pixels map and find oversensitive pixels
neighbour_ave_img = ndi.correlate(avg_img, neighbour_kernel)
mask = np.where(avg_img/neighbour_ave_img > neighbour_intensity_ratio, 1, mask)

plt.figure(figsize=(10,4))
ax1 = plt.subplot(1,3,1)
ax1.imshow(np.log(avg_img+3), interpolation='Nearest')
plt.title('log(average intensity)')
ax2 = plt.subplot(1,3,2, sharex=ax1, sharey=ax1)
ax2.imshow(mask, interpolation='Nearest')
plt.title('mask')
ax2 = plt.subplot(1,3,3, sharex=ax1, sharey=ax1)
ax2.imshow(avg_img/neighbour_ave_img, vmax=1.5, interpolation='Nearest')
plt.title('avg_int/avg_neighbour_int')

print(f'{np.sum(mask) - masked_px} was estimated to be oversensitive and has been masked.') 

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

5118 was estimated to be oversensitive and has been masked.


#### Diaplay details for single pixels

In [5]:
px_x, px_y = 1414, 1574

plt.figure()
for i in range(int(n_sigma+1)):
    plt.fill_between(np.linspace(0,300, 301), np.mean(images[:,px_y,px_x])+i*sqrt_avg_img[px_y,px_x], \
                 np.mean(images[:,px_y,px_x])-i*sqrt_avg_img[px_y,px_x], color='red', alpha=0.1)
plt.plot(images[:,px_y,px_x])
plt.title(f'Intensity of px [{px_x}, {px_y}]')
plt.ylabel('Intensity / A.U.)')
plt.xlabel('Exposure no.')

print(f'Number of outliers: {int(sum_mask[px_y, px_x])}')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Number of outliers: 1


#### Add masked pixels based on geometry

In [ ]:
# Mask settings
grow_edge_by = 2         # Number of pixels to mask near the edges between modules [2]
mask_edge_by = 2         # Number of pixels to mask along the outside edge of the detector [2]
grow_cdte_gap_by = 1     # Number of pixels to mask along the middle of each module wher the CdTe pieces leave a gap [1]

maskReadout = True      # Mask the edges of the individual readout chips [True]
grow_readout_by = 1     # Number of additional pixels to mask around the readout chips [1]

#Detector defaults
det_size = [1679, 1475]
module_size = [195, 487]
cdte_gap_pos = int((module_size[1]-1)/2)
gap_size = [17, 7]
module_no = [8, 3]
readout_chip_no = [2, 8]
readout_chip_size = [97, 60]
# Mask value
mask_value = 1
    
# Create horizontal masks
for i in range(module_no[0]-1):
    start = (module_size[0]+gap_size[0])*i+module_size[0]-grow_edge_by
    end = (module_size[0]+gap_size[0])*(i+1)-1+grow_edge_by
    mask[start:end+1, :] = mask_value

# Create vertical masks
for i in range(module_no[1]-1):
    start = (module_size[1]+gap_size[1])*i+module_size[1]-grow_edge_by
    end = (module_size[1]+gap_size[1])*(i+1)-1+grow_edge_by
    mask[:, start:end+1] = mask_value

# Mask mid-module (CdTe) gaps:
for i in range(module_no[1]):
    start = (module_size[1]+gap_size[1])*i+cdte_gap_pos-grow_cdte_gap_by
    end = (module_size[1]+gap_size[1])*i+cdte_gap_pos+grow_cdte_gap_by
    mask[:, start:end+1] = mask_value

# Mask between readout chips:
if maskReadout:
    # Create horizontal lines
    for i in range(module_no[0]):
        corner_module = (module_size[0]+gap_size[0])*i
        for j in range(readout_chip_no[0]-1):
            start = corner_module+(readout_chip_size[0]+1)*j+readout_chip_size[0]-grow_readout_by
            end = corner_module+(readout_chip_size[0]+1)*j+readout_chip_size[0]+grow_readout_by
            mask[start:end+1, :] = mask_value
    # Create vertical lines
    for i in range(module_no[1]):
        corner_module = (module_size[1]+gap_size[1])*i
        for j in range(readout_chip_no[1]-1):
            if j == 3:
                pass
            else:
                start = corner_module+(readout_chip_size[1]+1)*j+readout_chip_size[1]-grow_readout_by
                end = corner_module+(readout_chip_size[1]+1)*j+readout_chip_size[1]+grow_readout_by
                mask[:, start:end+1] = mask_value

# Mask edges:
if mask_edge_by > 0:
    mask[:, :mask_edge_by] = mask_value
    mask[:, -1*mask_edge_by:] = mask_value
    mask[:mask_edge_by, :] = mask_value
    mask[-1*mask_edge_by:, :] = mask_value

plt.figure()
plt.imshow(mask, interpolation='Nearest') 

#### Save mask for Azint (.npy)

In [9]:
fname_mask = fname.split('raw')[0]+'process/autogenerated_mask.npy'
np.save(fname_mask, mask, allow_pickle=True)

#### Save mask for Dioptas (.mask)

In [ ]:
from PIL import Image, ImageTk

fname_mask = fname.split('raw')[0]+'process/autogenerated.mask'
print(fname_mask)

im = Image.fromarray(np.asarray(mask, dtype=np.int32))
im.save(fname_mask, "TIFF")